# 流畅的Python / 20240111 <a class="tocSkip">

！！！🐍 由于原著版权原因，此学习笔记仅限个人交流学习目的，不做任何商业或广播用途。

""" 🐍 希望本书不仅仅是关于Python的，也是关于Python文化的。"""

# 第 1 章 Python数据模型

🐍 Python 最好的品质之一是一致性。

🐍 数据模型其实是对 Python 框架的描述，它规范了这门语言自身构建模块的接口，这些模块包括但不限于序列、迭代器、函数、类和上下文管理器。

🐍 不管在哪种框架下写程序，都会花费大量时间去实现那些会被框架本身调用的方法，Python 也不例外。Python 解释器碰到特殊的句法时，会使用特殊方法去激活一些基本的对象操作，这些特殊方法的名字以两个下划线开头，以两个下划线结尾（例如 __ getitem __ ）。比如 obj[key] 的背后就是 __ getitem __ 方法，为了能求得obj[key] 的值，解释器实际上会调用obj.__ getitem __ (key)。

🐍 这些特殊方法名能让你自己的对象实现和支持以下的语言构架，并与之交互：
• 迭代
• 集合类
• 属性访问
• 运算符重载
• 函数和方法的调用
• 对象的创建和销毁
• 字符串表示形式和格式化
• 管理上下文（即with 块）

## 一摞Python风格的纸牌

In [1]:
# 示例 c01_1_1_card.py, 一摞有序的纸牌
import collections

Card = collections.namedtuple('Card', ['rank', 'suit']) #具名元组，ref. §2.3.4

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA') # 2-10,JQKA, 13张牌
    suits = 'spades diamonds clubs hearts'.split() #梅花，方块，黑桃，红桃
    
    def __init__(self): #初始化类
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks] # 列表推导(列表表达式)，ref. §2.2 
    def __len__(self): #方法len()
        return len(self._cards)

    def __getitem__(self, position): #方法getitem()
        return self._cards[position]

🐍 自Python 2.6开始，namedtuple 就加入到 Python 里，用以构建只有少数属性但是没有方法的对象，比如数据库条目。

In [2]:
beer_card = Card('7', 'diamonds')
beer_card

Card(rank='7', suit='diamonds')

In [3]:
deck = FrenchDeck() #FrenchDeck类 的实例化 deck
len(deck) #方法len()，调用特殊方法 __len__()

52

In [4]:
# 第一张，最后一张
deck[0], deck[-1] #调用特殊方法 __getitem__()

(Card(rank='2', suit='spades'), Card(rank='A', suit='hearts'))

In [5]:
# 随机抽取一张纸牌
from random import choice

In [9]:
choice(deck) #调用特殊方法 __getitem__()

Card(rank='K', suit='hearts')

🐍 通过实现特殊方法来利用Python数据模型的两个好处：

1. 作为你的类的用户，他们不必去记住标准操作的各式名称(.size() 还是.length())

2. 可以更加方便地利用Python 的标准库，比如random.choice 函数，从而不用重新发明轮子。

🐍 因为__ getitem __ 方法把 [] 操作交给了self._cards 列表，所以我们的deck 类自动支持切
片（slicing）操作。

In [17]:
deck[:3]

[Card(rank='2', suit='spades'),
 Card(rank='3', suit='spades'),
 Card(rank='4', suit='spades')]

In [18]:
deck[12::13] #列表切片参数：start:end:interval

[Card(rank='A', suit='spades'),
 Card(rank='A', suit='diamonds'),
 Card(rank='A', suit='clubs'),
 Card(rank='A', suit='hearts')]

In [20]:
# 实现了__getitem__ 方法，这一摞牌就变成可迭代的了
for card in deck[:5]:
    print(card)

Card(rank='2', suit='spades')
Card(rank='3', suit='spades')
Card(rank='4', suit='spades')
Card(rank='5', suit='spades')
Card(rank='6', suit='spades')


In [22]:
# 反向迭代 reversed()
for card in reversed(deck[-5:]):
    print(card)

Card(rank='A', suit='hearts')
Card(rank='K', suit='hearts')
Card(rank='Q', suit='hearts')
Card(rank='J', suit='hearts')
Card(rank='10', suit='hearts')


🐍 迭代通常是隐式的，譬如说一个集合类型没有实现 __ contains __ 方法，那么 in 运算符就会按顺序做一次迭代搜索。

In [23]:
# in 运算符可以用在 FrenchDeck 类上，因为它是可迭代的
Card('Q', 'hearts') in deck

True

In [24]:
Card('7', 'beasts') in deck

False

In [25]:
# 排序：黑桃最大、红桃次之、方块再次、梅花最小
suit_values = dict(spades=3, hearts=2, diamonds=1, clubs=0)

def spades_high(card):
    rank_value = FrenchDeck.ranks.index(card.rank)
    return rank_value * len(suit_values) + suit_values[card.suit]

In [26]:
# sorted()方法排序
for card in sorted(deck, key=spades_high):
    print(card)

Card(rank='2', suit='clubs')
Card(rank='2', suit='diamonds')
Card(rank='2', suit='hearts')
Card(rank='2', suit='spades')
Card(rank='3', suit='clubs')
Card(rank='3', suit='diamonds')
Card(rank='3', suit='hearts')
Card(rank='3', suit='spades')
Card(rank='4', suit='clubs')
Card(rank='4', suit='diamonds')
Card(rank='4', suit='hearts')
Card(rank='4', suit='spades')
Card(rank='5', suit='clubs')
Card(rank='5', suit='diamonds')
Card(rank='5', suit='hearts')
Card(rank='5', suit='spades')
Card(rank='6', suit='clubs')
Card(rank='6', suit='diamonds')
Card(rank='6', suit='hearts')
Card(rank='6', suit='spades')
Card(rank='7', suit='clubs')
Card(rank='7', suit='diamonds')
Card(rank='7', suit='hearts')
Card(rank='7', suit='spades')
Card(rank='8', suit='clubs')
Card(rank='8', suit='diamonds')
Card(rank='8', suit='hearts')
Card(rank='8', suit='spades')
Card(rank='9', suit='clubs')
Card(rank='9', suit='diamonds')
Card(rank='9', suit='hearts')
Card(rank='9', suit='spades')
Card(rank='10', suit='clubs')
Ca

🐍 通过实现 __ len __ 和 __ getitem __ 这两个特殊方法，FrenchDeck 就跟一个Python 自有的序列数据类型一样，可以体现出Python 的核心语言特性（例如迭代和切片）。同时这个类还可以用于标准库中诸如 random.choice、reversed 和 sorted 这些函数。另外，对合成的运用使得 __ len __ 和 __ getitem __ 的具体实现可以代理给 self._cards 这个 Python 列表（即list 对象）。

## 如何使用特殊方法

🐍 特殊方法的存在是为了被Python 解释器调用的，你自己并不需要调用它们。没有my_object.__ len __ () 这种写法，而应该使用len(my_object)。

除非有大量的元编程 (§19-§21) 存在，直接调用特殊方法的频率应该远远低于你去实现它们的次数。唯一的例外可能是 __ init __ 方法，你的代码里可能
经常会用到它，目的是在你自己的子类的 __ init __ 方法中调用超类的构造器。

通过内置的函数（例如len、iter、str，等等）来使用特殊方法是最好的选择。

### 模拟数值类型

In [28]:
# 示例 c01_1_2_vector.py,　一个简单的二维向量类
from math import hypot

class Vector:
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

    def __repr__(self): #自定义 __repr__() 方法，字符串表示形式
        return 'Vector(%r, %r)' % (self.x, self.y)

    def __abs__(self): #自定义 __abs__() 方法，返回该向量的模
        return hypot(self.x, self.y) 

    def __bool__(self): #自定义 __bool__() 方法，返回布尔值
        return bool(abs(self))

    def __add__(self, other): #向量加法
        x = self.x + other.x
        y = self.y + other.y
        return Vector(x, y)

    def __mul__(self, scalar): #向量的标量乘法
        return Vector(self.x * scalar, self.y * scalar)

In [29]:
v1 = Vector(2, 4)
v2 = Vector(2, 1)

v1 + v2

Vector(4, 5)

In [30]:
v = Vector(3, 4)

abs(v)

5.0

### 字符串表示形式

🐍 Python 有一个内置的函数叫repr，它能把一个对象用字符串的形式表达出来以便辨认，这就是“字符串表示形式”。repr 就是通过 __ repr __ 这个特殊方法来得到一个对象的字符串表示形式的。

交互式控制台和调试程序（debugger）用repr 函数来获取字符串表示形式；在老的使用 % 符号的字符串格式中，这个函数返回的结果用来代替 %r 所代表的对象；同样，str.format 函数所用到的新式字符串格式化语法（https://docs.python.org/2/library/string.html#formatstring-
syntax）也是利用了repr，才把 !r 字段变成字符串。

🐍 __ repr __ 和 __ str __ 的区别在于，前者在Python解释器和调试程序时被repr()函数使用，后者是在str() 函数被使用，或是在用print 函数打印一个对象的时候才被调用的，并且它返回的字符串对终端用户更友好。

### 算术运算符

🐍 通过 __ add __ 和__ mul __ ，为向量类带来了 + 和 * 这两个算术运算符。

值得注意的是，这两个方法的返回值都是新创建的向量对象，被操作的两个向量（self 或other）还是原封不动，代码里只是读取了它们的值而已。

中缀运算符的基本原则就是不改变操作对象，而是产出一个新的值。

### 自定义的布尔值

🐍 为了判定一个值x 为真还是为假，Python会调用bool(x)，这个函数只能返回True 或者False。

bool(x) 的背后是调用 x.__ bool __ () 的结果；如果不存在 __ bool __ 方法，那么bool(x) 会尝试调用x.__ len __ ()。若返回0，则 bool 会返回False；否则返回True。

## 特殊方法一览

🐍 Python 语言参考手册中的“Data Model”（https://docs.python.org/3/reference/datamodel.html) 一章列出了83 个特殊方法的名字，其中47 个用于实现算术运算、位运算和比较操作。

![§1_3_dunder_method_1](c01_fig/§1_3_dunder_method_1.png)

![§1_3_dunder_method_2](c01_fig/§1_3_dunder_method_2.png)

## 为什么len不是普通方法

🐍 如果 x 是一个内置类型的实例，那么len(x) 的速度会非常快。背后的原因是CPython 会直接从一个C 结构体里读取对象的长度，完全不会调用任何方法。获取
一个集合中元素的数量是一个很常见的操作，在str、list、memoryview 等类型上，这个操作必须高效。

🐍 len 之所以不是一个普通方法，是为了让Python 自带的数据结构可以走后门，abs 也是同理。

## 本章小结

🐍 通过实现特殊方法，自定义数据类型可以表现得跟内置类型一样，从而让我们写出更具表达力的代码——或者说，更具Python 风格的代码。

🐍 Python对象的一个基本要求就是它得有合理的字符串表示形式，我们可以通过 __ repr __ 和 __ str __ 来满足这个要求。前者方便我们调试和记录日志，后者则是给终端用户看的。

🐍 对序列数据类型的模拟是特殊方法用得最多的地方

🐍 Python通过运算符重载这一模式提供了丰富的数值类型，除了内置的那些之外，还有decimal.Decimal 和fractions.Fraction。
这些数据类型都支持中缀算术运算符。

## 延伸阅读

🐍 对本章内容和本书主题来说，Python 语言参考手册里的“Data Model”一章（https://docs.python.org/3/reference/datamodel.html) 是最符合规范的知识来源。

🐍 Alex Martelli 的《Python 技术手册（第2 版）》对数据模型的讲解很精彩。
Martelli 还是Stack Overflow上的高产贡献者，在他名下差不多有5000 条答案，你也可以去他的Stack Overflow 主页（http://stackoverflow.com/users/95810/alex-martelli) 上看看。

🐍 David Beazley 著有两本基于Python 3 的书，其中对数据模型进行了详尽的介绍。
一本是《Python 参考手册（第4 版）》8，另一本是与Brian K. Jones 合著的《Python Cookbook（第3版）中文版》。

🐍 由Gregor Kiczales、Jim des Rivieres 和Daniel G. Bobrow 合著的The Art of the MetaobjectProtocol（ 又称AMOP，MIT 出版社，1991 年） 一书解释了元对象协议（metaobject
protocol，MOP）的概念，而Python 数据模型便是对这一概念的一种阐释。